In [19]:
import time

import pandas as pd
import dotenv
import os
import sys
import datetime
sys.path.append('./_01_Simulation/')

import _01_Simulation.LLM_Edge_Usability
import _01_Simulation.LLM_Dynamic_Weights
import _01_Simulation.LLM_MetaModel

In [20]:
# load .env file
dotenv.load_dotenv()

True

In [3]:
# Deactivate deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# 1. Edge usability

In [4]:
# Params (currently only considers openai models
approaches = ["zeroshot", "fewshot"]

In [5]:
# load evaluation csv
df_evaluation = pd.read_csv(os.path.join(os.getenv('RESOURCES'), 'EvaluationDataset.csv'), sep=";")

In [6]:
df_evaluation.head()

,action,edgeUsable
0,A person fell on the ground.,False
1,There is an unexpected construction on the usu...,False
2,Fallen branches from nearby trees.,False
3,Street cleaning crew sweeping debris.,False
4,The sidewalk is obstructed by a parked car.,False


In [7]:
# TODO Remove this statement to evaluate the whole dataset. This will induce extensive costs for API usage.
df_evaluation = df_evaluation.iloc[:2]

In [8]:
# Invoke GPT-3.5 to obtain the predictions for edge-usability

for approach in approaches:
    print(f"Approach: {approach}")
    predictions = []
    for index, row in df_evaluation.iterrows():
        model_output = _01_Simulation.LLM_Edge_Usability.invoke_llm(row["action"], model_type="openai", approach=approach)
        prediction = _01_Simulation.LLM_Edge_Usability.parse_response(model_output)
        predictions.append(prediction)
        print(f"Finished {index+1}/{len(df_evaluation)}")
    
    df_evaluation["prediction"] = predictions
    df_evaluation.to_csv(os.path.join(os.getenv('RESULTS'), f'eval-res-edge-usability-openai-{approach}-{datetime.datetime.now().isoformat()}.csv'), sep=";", index=False)

Approach: zeroshot
Finished 1/2
Finished 2/2
Approach: fewshot
Finished 1/2
Finished 2/2


# 2. Dynamic Edge Weights

In [9]:
# Params (currently only considers openai models
approaches = ["zeroshot", "fewshot"]

In [10]:
# load evaluation csv
df_evaluation = pd.read_csv(os.path.join(os.getenv('RESOURCES'), 'EvaluationDataset-dynamic-edge-weight.csv'), sep=";")

In [11]:
df_evaluation.head()

,action,lengthDependency,LLMAnswer
0,A person fell on the ground.,False,False
1,There is an unexpected construction on the usu...,True,True
2,Fallen branches from nearby trees.,False,False
3,Street cleaning crew sweeping debris.,False,False
4,The sidewalk is obstructed by a parked car.,False,False


In [12]:
# TODO Remove this statement to evaluate the whole dataset. This will induce extensive costs for API usage.
df_evaluation = df_evaluation.iloc[:2]

In [13]:
df_evaluation

,action,lengthDependency,LLMAnswer
0,A person fell on the ground.,False,False
1,There is an unexpected construction on the usu...,True,True


In [14]:
# Invoke GPT-3.5 to obtain the predictions for dynamic-edge-weights

for approach in approaches:
    print(f"Approach: {approach}")
    predictions = []
    result_types = []
    for index, row in df_evaluation.iterrows():
        model_output, _, result_type = _01_Simulation.LLM_Dynamic_Weights.invoke_llm_chain(row["action"], model_type="openai", approach=approach)
        prediction = _01_Simulation.LLM_Dynamic_Weights.parse_output_weights(model_output)
        predictions.append(prediction)
        result_types.append(result_type)
        print(f"Finished {index+1}/{len(df_evaluation)}")
    
    df_evaluation["prediction"] = predictions
    df_evaluation["result_type"] = result_types
    df_evaluation.to_csv(os.path.join(os.getenv('RESULTS'), f'eval-res-dynamic-openai-{approach}-{datetime.datetime.now().isoformat()}.csv'), sep=";", index=False)

Approach: zeroshot
Finished 1/2
Finished 2/2
Approach: fewshot
Finished 1/2
Finished 2/2


# 3. Functioncalling

i

# 4. Meta-Model

In [15]:
# load evaluation csv
df_evaluation = pd.read_csv(os.path.join(os.getenv('RESOURCES'), 'EvaluationDataset-metamodel.csv'), sep=";")

In [16]:
df_evaluation.head()

,examples
0,There is a fire alarm going off at node 31451751.
1,There is a power outage at node 5327549890.
2,There is a gas leak at node 2895270992.
3,An infectious disease outbreak was identified ...
4,There is a big medical conference being held a...


In [17]:
# TODO Remove this statement to evaluate the whole dataset. This will induce extensive costs for API usage.
df_evaluation = df_evaluation.iloc[:2]

In [18]:
output_usabilities = []
output_dynamics = []
output_lengths = []
output_times = []
output_nodes = []
output_nodes_times = []
for index, row in df_evaluation.iterrows():
    output_usability, output_dynamic, output_length, output_time, output_node, output_nodes_time, _ = _01_Simulation.LLM_MetaModel.invoke_llm(row["examples"])
    output_usabilities.append(output_usability)
    output_dynamics.append(output_dynamic)
    output_lengths.append(output_length)
    output_times.append(output_time)
    output_nodes.append(output_node)
    output_nodes_times.append(output_nodes_time)
    print(f"Finished {index+1}/{len(df_evaluation)}")

df_evaluation["output_usability"] = output_usabilities
df_evaluation["output_dynamic"] = output_dynamics
df_evaluation["output_length"] = output_lengths
df_evaluation["output_time"] = output_times
df_evaluation["output_node"] = output_nodes

df_evaluation.to_csv(os.path.join(os.getenv('RESULTS'), f'eval-res-metamodel-{datetime.datetime.now().isoformat()}.csv'), sep=";", index=False)

Finished 1/2
Finished 2/2
